In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Neuromatch_project/

/content/drive/MyDrive/Neuromatch_project


In [ ]:
!pip install torch torchvision matplotlib

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Training the model
If you want to test model, you can skip this part

In [ ]:
batch_size= 128
data_path= r'./data'
dtype= torch.float

# Create the transoform for MNIST dataset to make sure its 28x28, grayscale, a tensor, and vals normalized to fall between 0 and 1
transform= transforms.Compose([
    transforms.Resize((28,28)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,)),])

# Automatically downloads and splits the MNIST dataset
mnist_train = datasets.MNIST(data_path, train= True , download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train= False, download=True, transform=transform)

#create DataLoaders
train_loader= DataLoader(mnist_train , batch_size= batch_size, shuffle=True, drop_last=True)
test_loader= DataLoader(mnist_test , batch_size= batch_size, shuffle=True, drop_last=True)

## Model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = self.dropout1(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return x

In [ ]:
model = SimpleCNN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## training and testing function

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_bar = tqdm(train_loader, desc='Training')
    for batch_idx, (data, target) in enumerate(train_bar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        train_bar.set_postfix({
            'Loss': f'{running_loss/(batch_idx+1):.4f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })

    return running_loss / len(train_loader), 100. * correct / total

def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        test_bar = tqdm(test_loader, desc='Testing')
        for data, target in test_bar:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            test_bar.set_postfix({
                'Loss': f'{test_loss/len(test_loader):.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })

    return test_loss / len(test_loader), 100. * correct / total

## training process

In [ ]:
num_epochs = 10
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

print("\nStarting Training...")
start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch [{epoch+1}/{num_epochs}]")

    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)

    test_loss, test_acc = test_model(model, test_loader, criterion, device)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

end_time = time.time()
print(f"\nTraining completed in {end_time - start_time:.2f} seconds")


Starting Training...

Epoch [1/10]


Testing: 100%|██████████| 78/78 [00:02<00:00, 26.72it/s, Loss=0.0532, Acc=98.19%]


Train Loss: 0.2151, Train Acc: 93.19%
Test Loss: 0.0532, Test Acc: 98.19%

Epoch [2/10]


Testing: 100%|██████████| 78/78 [00:02<00:00, 31.18it/s, Loss=0.0327, Acc=98.91%]


Train Loss: 0.0604, Train Acc: 98.16%
Test Loss: 0.0327, Test Acc: 98.91%

Epoch [3/10]


...

Train Loss: 0.0197, Train Acc: 99.41%
Test Loss: 0.0200, Test Acc: 99.34%

Epoch [9/10]


Testing: 100%|██████████| 78/78 [00:02<00:00, 30.46it/s, Loss=0.0207, Acc=99.38%]


Train Loss: 0.0157, Train Acc: 99.53%
Test Loss: 0.0207, Test Acc: 99.38%

Epoch [10/10]


Testing: 100%|██████████| 78/78 [00:02<00:00, 29.41it/s, Loss=0.0206, Acc=99.33%]

Train Loss: 0.0169, Train Acc: 99.45%
Test Loss: 0.0206, Test Acc: 99.33%

Training completed in 238.16 seconds


In [ ]:
torch.save(model.state_dict(), 'small_cnn_model.pth')

# Test


## model, load parameter and dataset

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = self.dropout1(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return x

def load_model(model_path, device):
    """Load the saved model weights"""
    model = SimpleCNN(num_classes=10).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()  # Set to evaluation mode
    print(f"Model loaded from {model_path}")
    return model

batch_size = 128
data_path = r'./data'

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,)),
])

mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=False)

print(f"Test samples: {len(mnist_test)}")


Test samples: 10000


In [ ]:
model_path = 'small_cnn_model.pth' ## change this to the path of weight
model = load_model(model_path, device)

Model loaded from small_cnn_model.pth


## model accuracy on test dataset

In [ ]:
def comprehensive_test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        test_bar = tqdm(test_loader, desc='Testing')
        for data, target in test_bar:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()

            c = (predicted == target).squeeze()
            for i in range(target.size(0)):
                label = target[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(target.cpu().numpy())

            test_bar.set_postfix({'Acc': f'{100.*correct/total:.2f}%'})

    overall_accuracy = 100. * correct / total

    print(f"\nOverall Test Accuracy: {overall_accuracy:.2f}%")
    print("\nPer-class Accuracy:")
    for i in range(10):
        if class_total[i] > 0:
            accuracy = 100 * class_correct[i] / class_total[i]
            print(f'Class {i}: {accuracy:.2f}')

    return overall_accuracy, all_predictions, all_labels

In [ ]:
accuracy_test, all_predictions, all_labels = comprehensive_test(model, test_loader, device)

Testing: 100%|██████████| 79/79 [00:03<00:00, 24.35it/s, Acc=99.33%]


Overall Test Accuracy: 99.33%

Per-class Accuracy:
Class 0: 99.80
Class 1: 99.82
Class 2: 99.81
Class 3: 99.50
Class 4: 99.59
Class 5: 99.33
Class 6: 98.85
Class 7: 99.12
Class 8: 99.38
Class 9: 98.02
